## 1. pandas version

In [1]:
!python --version

Python 3.7.15


In [ ]:
# !pip install pandas numpy tqdm

In [4]:
!pip3 install -U scikit-learn

  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (38.1 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [45]:
import pandas as pd
import numpy as np
from time import time
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsRegressor

In [68]:
import random
import string

In [5]:
def get_random_string(length):
    # With combination of lower and upper case
    result_str = ''.join(random.choice(string.ascii_letters) for i in range(length))
    # print random string
    return result_str

# string of length 8


In [28]:
user_ratings = pd.read_csv("user_ratings.csv")

In [29]:
user_ratings_table = user_ratings.pivot(index='userId', columns='movieId', values='rating')
user_ratings_table.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
user_ratings_table = user_ratings_table[user_ratings_table.columns.tolist()[:300]]

In [31]:
# fake idsn
dict_isdn = {}
list_isdn = []
for i in range(len(user_ratings_table.index)):
    dict_isdn[str(i)] = get_random_string(8)
    list_isdn.append(get_random_string(8))
# print(dict_isdn)

In [32]:
user_ratings_table = user_ratings_table.reset_index(drop=True)


In [33]:
user_ratings_table['isdn'] = list_isdn

In [34]:
user_ratings_table = user_ratings_table.set_index('isdn')


In [35]:
user_ratings_table

movieId,1,2,3,4,5,6,7,8,9,10,...,332,333,334,335,336,337,338,339,340,341
isdn,,,,,,,,,,,,,,,,,,,,,
JTHlOjRY,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
oMUOOmvO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CLmZiPfi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BPpkyQiw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aFikAaLO,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fksHLfrH,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
JcsmidqP,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
PYVdSgUW,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN


In [36]:
list_movie = user_ratings_table.columns
list_movie = list_movie.tolist()

In [37]:
tmp_table =user_ratings_table.fillna(0)

In [38]:
# Get the average rating for each user 
avg_ratings = user_ratings_table.mean(axis=1)
# Center each users ratings around 0
user_ratings_table_centered = user_ratings_table.sub(avg_ratings, axis=0)
# Fill in the missing data with 0s
user_ratings_table_normed = user_ratings_table_centered.fillna(0)

In [39]:
user_ratings_table_normed

movieId,1,2,3,4,5,6,7,8,9,10,...,332,333,334,335,336,337,338,339,340,341
isdn,,,,,,,,,,,,,,,,,,,,,
JTHlOjRY,-0.315789,0.000000,-0.315789,0.0,0.0,-0.315789,0.00000,0.0,0.0,0.000000,...,0.0000,0.684211,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
oMUOOmvO,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,...,0.0000,0.500000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
CLmZiPfi,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,...,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
BPpkyQiw,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,...,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
aFikAaLO,0.380952,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,...,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fksHLfrH,-1.182540,0.000000,0.000000,0.0,0.0,0.000000,-1.18254,0.0,0.0,0.000000,...,0.0000,0.000000,0.0,0.0,0.0,0.317460,0.0,0.000000,0.0,0.0
JcsmidqP,0.380952,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,...,0.0000,0.000000,0.0,0.0,0.0,-0.619048,0.0,0.000000,0.0,0.0
PYVdSgUW,-0.295455,-0.795455,-0.795455,0.0,0.0,0.000000,0.00000,0.0,0.0,1.204545,...,0.0000,1.204545,0.0,0.0,0.0,0.000000,0.0,-0.295455,0.0,0.0


In [40]:
similarities = cosine_similarity(user_ratings_table_normed)
cosine_similarity_df = pd.DataFrame(similarities, index=user_ratings_table_normed.index, columns=user_ratings_table_normed.index)
cosine_similarity_df

isdn,JTHlOjRY,oMUOOmvO,CLmZiPfi,BPpkyQiw,aFikAaLO,gugZIWbH,UcXxhuen,AGTcJLsF,rLsOLOiP,NbGvOcYA,...,nynaBels,uHnFDqha,HwlqoklY,mjbEdBtJ,uYLtrtgO,fksHLfrH,JcsmidqP,PYVdSgUW,IjaClTSm,QiUNndNZ
isdn,,,,,,,,,,,,,,,,,,,,,
JTHlOjRY,1.000000,0.139055,0.0,0.156139,0.020930,-3.671859e-02,0.112882,0.086210,-0.308783,0.0,...,0.045382,0.011440,0.007102,-0.155242,0.321444,-0.153499,0.048249,0.020853,-0.167390,-0.044622
oMUOOmvO,0.139055,1.000000,0.0,0.000000,0.095630,3.039091e-19,0.000000,-0.219701,0.000000,0.0,...,-0.353553,-0.145144,0.000000,0.000000,0.000000,0.025738,-0.252527,-0.040692,-0.309173,0.225172
CLmZiPfi,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BPpkyQiw,0.156139,0.000000,0.0,1.000000,-0.146997,2.451749e-02,0.157414,-0.001576,0.000000,0.0,...,0.096372,-0.078839,0.051568,-0.068237,0.313773,0.069514,0.014896,-0.123281,-0.148207,-0.070177
aFikAaLO,0.020930,0.095630,0.0,-0.146997,1.000000,-7.132499e-02,-0.239641,0.044487,0.000000,0.0,...,-0.067620,-0.065073,0.111939,0.029434,-0.264405,0.149224,-0.033822,0.090836,0.009447,0.172788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fksHLfrH,-0.153499,0.025738,0.0,0.069514,0.149224,-8.967288e-02,0.097041,0.006030,0.133090,0.0,...,0.249249,0.135116,0.035350,0.307355,-0.114122,1.000000,-0.093350,0.173889,0.158774,0.163013
JcsmidqP,0.048249,-0.252527,0.0,0.014896,-0.033822,1.126187e-01,-0.019977,0.082163,0.000000,0.0,...,0.129305,0.124393,-0.095755,-0.130616,0.099068,-0.093350,1.000000,0.040460,-0.014911,-0.060061
PYVdSgUW,0.020853,-0.040692,0.0,-0.123281,0.090836,-2.021785e-02,0.050921,0.115581,-0.027765,0.0,...,0.115096,0.159215,0.005696,0.011924,-0.187964,0.173889,0.040460,1.000000,0.259187,0.131479


In [ ]:
user_ratings_table_normed.loc[['MESfDtdA']]

In [ ]:
user_ratings_table[2]

In [41]:
def fill_missing_score(cat_id, isdn, user_ratings_table_normed):
    '''
    loại bỏ cột y cần dự đoán cho người dùng x
    return Score tại cột y, hàng x
    '''
    # print(x)
    # print("cvat & isdn",cat_id, isdn)
    t1 = time()
    tmp_user_ratings_table_normed = user_ratings_table_normed
    # Drop the column you are trying to predict
    tmp_user_ratings_table_normed = tmp_user_ratings_table_normed.drop(cat_id, axis=1)
    # Get the data for the user you are predicting for
    target_user_x = tmp_user_ratings_table_normed.loc[[isdn]]
    # print(target_user_x.shape)
    # Get the target data from user_ratings_table
    other_users_y = user_ratings_table[cat_id]
    # print(other_users_y)
    # Get the data for only those that have seen the movie 
    other_users_x = tmp_user_ratings_table_normed[other_users_y.notnull()]
    # print(other_users_y.notnull().shape)
    # Remove those that have not seen the movie from the target 
    other_users_y.dropna(inplace=True)
    # print(user_ratings_table_normed.shape)
    # print(other_users_x)
    print(target_user_x.shape, other_users_x.shape, other_users_y.shape)

    # Instantiate the user KNN model
    user_knn = KNeighborsRegressor(metric='cosine', n_neighbors=7)
    # Fit the model and predict the target user
    user_knn.fit(other_users_x, other_users_y)
    user_user_pred = user_knn.predict(target_user_x)
    # print(time()-t1)
    return user_user_pred

In [1]:
tmp_table

NameError: name 'tmp_table' is not defined

In [45]:
for col_head, col in tmp_table.iteritems():
    print(col_head)
    for row, value in col.iteritems():
        print(row)
        print(value)
        break
    break

1
RYWPGYgk
4.0


In [ ]:
fill_missing_score(187, 3, user_ratings_table_normed)

In [42]:
# Instantiate the user KNN model
n_neighbor = 10
user_knn = KNeighborsRegressor(metric='cosine', n_neighbors=n_neighbor)
# for icol in tqdm(range(0, len(tmp_table))):
for col_head, col in tqdm(tmp_table.iteritems(), total=tmp_table.shape[0]):
  '''
  modified by cell iter
  col: col series ~ col
  col_head: index col ~ cat_id
  '''
  # cat_id = list_movie[icol]
  # print(cat_id)
  # col = tmp_table.iloc[:,icol]
  # print(col)
  tmp_user_ratings_table_normed = user_ratings_table_normed
  # Drop the column you are trying to predict
  tmp_user_ratings_table_normed = tmp_user_ratings_table_normed.drop(col_head, axis=1)
  other_users_y = user_ratings_table[col_head]
  other_users_x = tmp_user_ratings_table_normed[other_users_y.notnull()]
  other_users_y.dropna(inplace=True)
  user_knn.fit(other_users_x, other_users_y)
  if len(other_users_y) < n_neighbor:
    continue      
  
  # for irow, score in enumerate(col, 0):
  for isdn, score in col.iteritems():
    '''
    isdn: index row
    score: value cell
    '''
        
    # isdn = str(list_isdn[irow])
    target_user_x = tmp_user_ratings_table_normed.loc[[isdn]]
    user_user_pred = user_knn.predict(target_user_x)
    # print(cat_id, user_id, type(cat_id))
    if score == 0:
      # predict_value = fill_missing_score(cat_id, isdn, user_ratings_table_normed)
      predict_value = user_user_pred
      # print('predict_value:', predict_value)
      # modified at irow, icol
      # print("index ",isdn, col_head)
      tmp_table.at[isdn, col_head] = predict_value
      # break
    # break
  print('Filtered Done!')

 49%|████▉     | 300/610 [07:02<07:16,  1.41s/it]


In [44]:
tmp_table.to_csv('result_CF1.csv')

### Evaluate

In [ ]:
#Fake data

In [64]:
def count_positive(df):
    num_positive = df[df>0].count().sum()
    total_value = df[df>=0].count().sum()
    print(num_positive, total_value)

In [208]:
# results of CF predict
pred_table = pd.read_csv("result_CF1.csv", index_col='isdn')
count_positive(pred_table)

103581 183000


In [67]:
103581/183000

0.566016393442623

In [209]:
# Generate result in time T2
user_ratings = pd.read_csv("user_ratings.csv")
user_ratings_table = user_ratings.pivot(index='userId', columns='movieId', values='rating')
user_ratings_table = user_ratings_table[user_ratings_table.columns.tolist()[:300]]
user_ratings_table['isdn'] = list_isdn
user_ratings_table = user_ratings_table.set_index('isdn')
user_ratings_table = user_ratings_table.fillna(0)

In [210]:
old_table = user_ratings_table

In [212]:
count_positive(old_table)

9444 183000


In [205]:
gt_table = old_table

In [206]:

# fake more data for gt table
mutation = 0.4
for col_head, col in tqdm(gt_table.iteritems(), total=gt_table.shape[0]):
  '''
  modified by cell iter
  col: col series ~ col
  col_head: index col ~ cat_id
  '''
  for isdn, score in col.iteritems():
    '''
    isdn: index row
    score: value cell
    '''
    r = random.random()
    if score == 0 & (r>mutation): 
      if r>mutation:
        predict_value = round(random.uniform(0.0, 5.0),2)
        # print(r>0.4)
        gt_table.at[isdn, col_head] = predict_value
    # break
  # break
print('Filtered Done!')

 49%|████▉     | 300/610 [00:01<00:01, 276.05it/s]

Filtered Done!


In [ ]:
gt_table

In [207]:
count_positive(gt_table)

113555 183000


In [178]:
#  Evaluate
pred_table
gt_table
old_table

True

In [ ]:
bool_old_value = old_table>1
bool_old_value

In [ ]:
# filter old value =0 
pred_table.where(bool_old_value, 0)
gt_table.where(bool_old_value, 0)

In [223]:
pred_table.to_csv("pred_table.csv")
gt_table.to_csv("gt_table.csv")

,0
0,0
1,0
2,0
3,4
4,5


## 2. Pyspark verison

In [2]:
!pip install pyspark==2.4.4 py4j==0.10.7 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 MB 4.3 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 5.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130374 sha256=1feea6ca20859f20eef4f44cf9a608470dce341f79d08027db53285209b1959e
  Stored in directory: /home/admin1/.cache/pip/wheels/f8/2a/7b/79b37847f55a0f50af9562409d1cad82753a7481652bab8534
Successfully built pyspark


In [15]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("VeryFistSparkExample").getOrCreate()
spark.createDataFrame([('Java','20000'),('python','10000'),('scala','50000')]).show()

+------+-----+
|    _1|   _2|
+------+-----+
|  Java|20000|
|python|10000|
| scala|50000|
+------+-----+



In [16]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [6]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 803.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 2.0 MB/s eta 0:00:0000:0100:01


In [7]:

# importing pandas as pd
import pandas as pd
 
# Creating the dataframe
df = pd.DataFrame({"A":[1, 5, 3, 4, 2],
                   "B":[3, 2, 4, 3, 4],
                   "C":[2, 2, 7, 3, 4],
                   "D":[4, 3, 6, 12, 7]},
                   index =["A1", "A2", "A3", "A4", "A5"])
 
# Print the dataframe
df

,A,B,C,D
A1,1,3,2,4
A2,5,2,2,3
A3,3,4,7,6
A4,4,3,3,12
A5,2,4,4,7


In [11]:
avg = df.mean(axis=1)
 
# Print the series
avg

A1    2.50
A2    3.00
A3    5.00
A4    5.50
A5    4.25
dtype: float64

In [14]:

# equivalent to df - sr
df.sub(avg, axis = 0)

,A,B,C,D
A1,-1.50,0.50,-0.50,1.50
A2,2.00,-1.00,-1.00,0.00
A3,-2.00,-1.00,2.00,1.00
A4,-1.50,-2.50,-2.50,6.50
A5,-2.25,-0.25,-0.25,2.75
